In [6]:
import requests
import json
from urllib import parse
import os
import time
import pandas as pd  # 导入pandas库，用来读取和保存Excel文件
from openpyxl import Workbook
from openpyxl.drawing.image import Image


class BaiduImageSpider(object):
    def __init__(self):
        self.json_count = 1  # 每次请求获取1个json文件（即30张图片）
        self.url = 'https://image.baidu.com/search/acjson?tn=resultjson_com&logid=5179920884740494226&ipn=rj&ct' \
                   '=201326592&is=&fp=result&queryWord={}&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=-1&z=&ic=0&hd=1' \
                   '&latest=&copyright=&word={}&s=&se=&tab=&width=&height=&face=0&istype=2&qc=&nc=1&fr=&expermode=' \
                   '&nojc=&pn={}&rn=30&gsm=1e&1635054081427='  # 百度图片搜索的URL模板
        self.header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/95.0.4638.54 Safari/537.36 Edg/95.0.1020.30 '
        }
        self.base_directory = os.path.join(os.getcwd(), "智库下载图片2")  # 创建一个总文件夹，存放所有图片
        self.folder_counter = 1  # 用于生成按顺序编号的文件夹
        self.image_paths = []  # 用来存储每个关键词的下载图片路径

    # 创建总文件夹下的子文件夹，使用顺序编号和查询关键词作为文件夹名
    def create_directory(self, name):
        # 创建总文件夹（如果不存在）
        if not os.path.exists(self.base_directory):
            os.makedirs(self.base_directory)

        # 使用编号和关键词来创建文件夹
        folder_name = f"{str(self.folder_counter).zfill(2)}_{name}"  # 如：01_关键词
        self.directory = os.path.join(self.base_directory, folder_name)

        # 创建文件夹
        if not os.path.exists(self.directory):
            os.makedirs(self.directory)

        # 更新文件夹编号
        self.folder_counter += 1

    # 获取图像链接，调整为只获取1张高清图片
    def get_image_link(self, url):
        list_image_link = []
        try:
            strhtml = requests.get(url, headers=self.header)
            strhtml.raise_for_status()  # 检查HTTP请求是否成功
            jsonInfo = json.loads(strhtml.text)
            if 'data' in jsonInfo and jsonInfo['data']:
                # 只取第一张图片
                image_data = jsonInfo['data'][0]
                if 'thumbURL' in image_data:  # 确保有图片链接
                    list_image_link.append(image_data['thumbURL'])
        except requests.exceptions.RequestException as e:
            print(f"请求出错: {e}")
        return list_image_link

    # 下载图片
    def save_image(self, img_link, filename):
        try:
            res = requests.get(img_link, headers=self.header)
            res.raise_for_status()  # 检查HTTP请求是否成功
            with open(filename, "wb") as f:
                f.write(res.content)
                print(f"图片已保存到: {filename}")
            return filename  # 返回保存的图片路径
        except requests.exceptions.RequestException as e:
            print(f"图片下载出错: {e}")
            return None

    # 处理每个关键词进行图片下载
    def run(self, keywords, original_excel_file):
        results = []  # 用来存储每个关键词的下载结果
        for searchName in keywords:
            searchName_parse = parse.quote(searchName)  # 对关键词进行编码

            # 创建以查询内容为文件夹名的子目录，并添加编号
            self.create_directory(searchName)

            pic_number = 0  # 图像数量
            found_images = False  # 标记是否找到图片
            image_paths = []  # 用于存储当前关键词的图片路径

            try:
                for index in range(self.json_count):
                    pn = (index) * 30  # 调整分页参数为1张图片
                    request_url = self.url.format(searchName_parse, searchName_parse, str(pn))
                    list_image_link = self.get_image_link(request_url)

                    if list_image_link:  # 如果找到了图片链接
                        found_images = True
                        link = list_image_link[0]  # 只取第一张图片
                        pic_number += 1
                        # 存储图片到关键词文件夹
                        image_path = self.save_image(link, os.path.join(self.directory, f'{pic_number}.jpg'))
                        if image_path:
                            image_paths.append(image_path)  # 将下载的图片路径添加到列表
                        time.sleep(0.5)  # 休眠0.5秒，防止封IP

                # 记录下载结果
                if found_images:
                    results.append([searchName] + image_paths)  # 存储关键词和图片路径
                    print(f"{searchName} ---- 图像下载完成--------->")
                else:
                    results.append([searchName, '未找到图片'])  # 如果未找到图片
                    print(f"{searchName} ---- 未找到图片，已跳过。")

            except Exception as e:
                # 捕获所有异常并处理
                print(f"错误处理：{searchName} - {e}")
                # 即使发生错误，也将结果记录为未找到图片并跳过
                results.append([searchName, '下载失败'])
                print(f"{searchName} ---- 发生错误，跳过该关键词。")

        # 保存结果到Excel文件并插入图片
        self.save_to_excel(results, original_excel_file)

    # 将结果保存到Excel文件并插入图片
    def save_to_excel(self, results, original_excel_file):
        # 读取原始Excel文件
        df = pd.read_excel(original_excel_file)

        # 创建一个新的Excel工作簿
        wb = Workbook()
        ws = wb.active

        # 设置标题行，保留原始数据列
        header = df.columns.tolist() + ['图片1']
        ws.append(header)

        # 遍历每个关键词的结果
        for i, result in enumerate(results):
            keyword = result[0]
            image_paths = result[1:]

            # 获取原始文件中的一行数据
            original_row = df.iloc[i].tolist()

            # 向Excel中插入原始数据和图片路径
            ws.append(original_row + [''] * (5 - len(image_paths)))  # 保证每个关键词都有1个单元格

            # 向单元格中插入图片
            if image_paths:
                image_path = image_paths[0]  # 取第一张图片
                if image_path and os.path.exists(image_path):
                    img = Image(image_path)
                    img.width = 100  # 设置图片宽度
                    img.height = 100  # 设置图片高度
                    cell = ws.cell(row=ws.max_row, column=len(original_row) + 1)  # 图片插入到对应列
                    ws.add_image(img, cell.coordinate)

        # 保存最终结果到Excel文件
        output_filename = f"4测试.xlsx"
        wb.save(output_filename)
        print(f"所有结果已保存到 '{output_filename}' 文件中。")


if __name__ == '__main__':
    # 读取Excel文件中的关键词列表
    excel_file = r"C:\Users\86155\Desktop\学习\6.xlsx"  # 替换为你的Excel文件路径
    df = pd.read_excel(excel_file)  # 读取Excel文件
    keywords = df.iloc[:, 0].tolist()  # 假设关键词存储在第三列

    spider = BaiduImageSpider()
    spider.json_count = 1  # 设置每次只下载1张图片
    spider.run(keywords, excel_file)  # 传入从Excel读取的关键词列表


错误处理：加勒比地区 300 万美元的豪宅 - Invalid \escape: line 54 column 148 (char 63844)
加勒比地区 300 万美元的豪宅 ---- 发生错误，跳过该关键词。
图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\智库下载图片2\02_加勒比辣椒酱如何火起来\1.jpg
加勒比辣椒酱如何火起来 ---- 图像下载完成--------->
图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\智库下载图片2\03_巨型船舶之战？一艘欧洲巨无霸驶向加勒比海。\1.jpg
巨型船舶之战？一艘欧洲巨无霸驶向加勒比海。 ---- 图像下载完成--------->
错误处理：从加利福尼亚到加勒比海：5 个阳光明媚的新度假胜地 - Invalid \escape: line 14 column 158 (char 19069)
从加利福尼亚到加勒比海：5 个阳光明媚的新度假胜地 ---- 发生错误，跳过该关键词。
图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\智库下载图片2\05_飓风席卷加勒比地区\1.jpg
飓风席卷加勒比地区 ---- 图像下载完成--------->
图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\智库下载图片2\06_古根海姆策展人将举办加勒比侨民展览\1.jpg
古根海姆策展人将举办加勒比侨民展览 ---- 图像下载完成--------->
错误处理：克里斯托弗·哥伦布是否引发了气候危机？ - Invalid control character at: line 84 column 157 (char 42949)
克里斯托弗·哥伦布是否引发了气候危机？ ---- 发生错误，跳过该关键词。
图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\智库下载图片2\08_邮轮公司改变行程以避开飓风海伦的影响\1.jpg
邮轮公司改变行程以避开飓风海伦的影响 ---- 图像下载完成--------->
图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\智库下载图片2\09_想看珊瑚礁生长吗？那就冷冻它们吧。\1.jpg
想看珊瑚礁生长吗？那就冷冻它们吧。 ---- 图像下载完成--------->
图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\智库下载图片2\10_飓风贝丽尔袭击

In [14]:
import requests
import json
from urllib import parse
import os
import time
import pandas as pd
from openpyxl import Workbook
from openpyxl.drawing.image import Image
import logging
import re
# 配置日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


class BaiduImageSpider(object):
    def __init__(self):
        self.json_count = 1  # 每次请求获取1个json文件（即1张图片）
        self.url = 'https://image.baidu.com/search/acjson?tn=resultjson_com&logid=5179920884740494226&ipn=rj&ct=201326592&is=&fp=result&queryWord={}&cl=2&lm=-1&ie=utf-8&oe=utf-8&adpicid=&st=-1&z=&ic=0&hd=1&latest=&copyright=&word={}&s=&se=&tab=&width=&height=&face=0&istype=2&qc=&nc=1&fr=&expermode=&nojc=&pn={}&rn=30&gsm=1e&1635054081427='
        self.header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36 Edg/95.0.1020.30'
        }
        self.base_directory = os.path.join(os.getcwd(), "下载图片")  # 创建一个总文件夹，存放所有图片
        self.folder_counter = 1  # 用于生成按顺序编号的文件夹

    def clean_filename(self, filename):
        # 替换Windows不允许的字符
        filename = re.sub(r'[<>:"/\\|?*]', '_', filename)
        # 确保文件名不以点开头
        if filename.startswith('.'):
            filename = '_' + filename
        # 确保文件名长度不超过Windows限制（通常为255个字符）
        if len(filename) > 255:
            filename = filename[:255]
        return filename
    # 创建总文件夹下的子文件夹    
    def create_directory(self, name):
        if not os.path.exists(self.base_directory):
            os.makedirs(self.base_directory)
        clean_name = self.clean_filename(name)
        folder_name = f"{str(self.folder_counter).zfill(2)}_{clean_name}"  # 如：01_关键词
        self.directory = os.path.join(self.base_directory, folder_name)
        if not os.path.exists(self.directory):
            os.makedirs(self.directory)
        self.folder_counter += 1

    # 获取图像链接
    def get_image_link(self, url):
        try:
            response = requests.get(url, headers=self.header)
            response.raise_for_status()
            json_info = json.loads(response.text)
            if 'data' in json_info and json_info['data']:
                image_data = json_info['data'][0]
                if 'thumbURL' in image_data:
                    return [image_data['thumbURL']]
        except requests.exceptions.RequestException as e:
            logging.error(f"请求出错: {e}")
        except json.JSONDecodeError as e:
            logging.error(f"JSON解析错误: {e}")
        return []

    # 下载图片
    def save_image(self, img_link, filename):
        try:
            response = requests.get(img_link, headers=self.header)
            response.raise_for_status()
            with open(filename, "wb") as f:
                f.write(response.content)
            logging.info(f"图片已保存到: {filename}")
            return filename
        except requests.exceptions.RequestException as e:
            logging.error(f"图片下载出错: {e}")
        return None

    # 处理每个关键词进行图片下载
    def process_keyword(self, searchName):
        searchName_parse = parse.quote(searchName)
        self.create_directory(searchName)
        try:
            request_url = self.url.format(searchName_parse, searchName_parse, "0")
            image_links = self.get_image_link(request_url)
            if image_links:
                img_link = image_links[0]
                image_path = self.save_image(img_link, os.path.join(self.directory, f'1.jpg'))
                if image_path:
                    return [searchName, image_path]
        except Exception as e:
            logging.error(f"处理关键词 {searchName} 时出错: {e}")
        return [searchName, "错误"]

    # 将结果保存到Excel文件并插入图片
    def save_to_excel(self, results, original_excel_file):
        df = pd.read_excel(original_excel_file)
        wb = Workbook()
        ws = wb.active
        header = df.columns.tolist() + ['图片路径']
        ws.append(header)

        for i, result in enumerate(results):
            keyword = result[0]
            image_path = result[1]
            original_row = df.iloc[i].tolist()
            ws.append(original_row + [image_path])

            if image_path != "错误" and os.path.exists(image_path):
                img = Image(image_path)
                img.width = 100
                img.height = 100
                cell = ws.cell(row=i + 2, column=len(header))
                ws.add_image(img, cell.coordinate)

        output_filename = "外交部带图片.xlsx"
        wb.save(output_filename)
        logging.info(f"所有结果已保存到 '{output_filename}' 文件中。")

    # 主运行函数
    def run(self, keywords, original_excel_file):
        results = []
        for searchName in keywords:
            result = self.process_keyword(searchName)
            results.append(result)
            logging.info(f"{searchName} 处理完成")
        self.save_to_excel(results, original_excel_file)


if __name__ == '__main__':
    excel_file = r"D:\学习\Naraka\加勒比地区数据\外交部.xlsx"  # 替换为你的Excel文件路径
    df = pd.read_excel(excel_file)
    keywords = df.iloc[:, 3].tolist()  # 假设关键词存储在第一列

    spider = BaiduImageSpider()
    spider.run(keywords, excel_file)

2025-03-08 13:23:36,546 - INFO - 图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\下载图片\01_中国和古巴两国外交部签署档案领域合作协议\1.jpg
2025-03-08 13:23:36,548 - INFO - 中国和古巴两国外交部签署档案领域合作协议 处理完成
2025-03-08 13:23:36,802 - INFO - 图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\下载图片\02_王沪宁会见古巴全国人民政权代表大会主席拉索\1.jpg
2025-03-08 13:23:36,803 - INFO - 王沪宁会见古巴全国人民政权代表大会主席拉索 处理完成
2025-03-08 13:23:37,111 - INFO - 图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\下载图片\03_王毅会见古巴外长罗德里格斯\1.jpg
2025-03-08 13:23:37,113 - INFO - 王毅会见古巴外长罗德里格斯 处理完成
2025-03-08 13:23:37,454 - INFO - 图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\下载图片\04_中国政府拉美事务特别代表邱小琪出席古巴驻华使馆庆祝中古建交64周年暨双边教育合作60周年招待会\1.jpg
2025-03-08 13:23:37,456 - INFO - 中国政府拉美事务特别代表邱小琪出席古巴驻华使馆庆祝中古建交64周年暨双边教育合作60周年招待会 处理完成
2025-03-08 13:23:37,689 - INFO - 图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\下载图片\05_外交部部长助理苗得雨会见古巴外交部政策规划总司代总司长阿奇基纳西\1.jpg
2025-03-08 13:23:37,690 - INFO - 外交部部长助理苗得雨会见古巴外交部政策规划总司代总司长阿奇基纳西 处理完成
2025-03-08 13:23:37,959 - INFO - 图片已保存到: D:\我的文档\我的代码\爬虫\爬虫\下载图片\06_李希会见古巴共产党省市委领导干部考察团\1.jpg
2025-03-08 13:23:37,961 - INFO - 李希会见古巴共产党省市委领导干部考察团 处理完成
2025-03-08 1